Note, this requires using PyTorch v0.3.1.  Somewhere between 0.3.1 and 0.4.0 parts of the backend were significantly rewritten, preventing us from performing the following hacks. (Likely has to do with them fusing Variable and Tensor).

In [1]:
import torch
from torch.autograd import Variable
import inspect
#from forbiddenfruit import curse

In [2]:
legal_commands = ['Variable.add', 'Variable.__add__', 'Variable.add_', 'Variable.__iadd__','addmm']

In [3]:
def dummy(x, y , z = 2):
    frame = inspect.currentframe()
    args, varargs, keywords, values = inspect.getargvalues(frame)
    print(args)
    print(varargs)
    print(keywords)
    print(values)
    print(values['frame'])

In [4]:
dummy(1, 2, z = 5)

['x', 'y', 'z']
None
None
{'frame': <frame object at 0x10fb33048>, 'z': 5, 'y': 2, 'x': 1}


In [5]:
def compile_command(name, frame):
    args, _, _, values = inspect.getargvalues(frame)
    command = {}
    command['command'] = name
    command['args'] = args
    command['values'] = [values[arg] for arg in args]
    command['types'] = [type(val) for val in command['values']]
    return command

In [6]:
x = Variable(torch.FloatTensor([1,2,3,4]),requires_grad=True)

In [7]:
inspect.getinnerframes()

TypeError: getinnerframes() missing 1 required positional argument: 'tb'

In [25]:
inspect.getargvalues(frame)

NameError: name 'frame' is not defined

In [21]:
x.__class__.__name__

'Variable'

In [23]:
x.add.__name__

'add'

In [24]:
x.add(y)

Variable containing:
 3
 5
 7
 9
[torch.FloatTensor of size 4]

In [13]:
compile_command('add', )

TypeError: compile_command() takes 2 positional arguments but 3 were given

In [3]:
def send_command(command, *args, **kwargs):
    assert type(command) == str

In [ ]:
a, b = Var(x), Var(y)
a + b != Var(x + y)
Variable.__add__ != FloatTensor.__add__

In [2]:
def hook_var():
    
    Variable.lit___add__ = Variable.__add__ 
    def grid___add__(self,other):
        print("__add__")
        return self.lit___add__(other)
    Variable.__add__ = grid___add__
    
    Variable.lit_add = Variable.add
    def grid_add(self,other):
        print("add")
        return self.lit_add(other)
    Variable.add = grid_add

def hook_float():
    torch.FloatTensor.lit___add__ = torch.FloatTensor.__add__ 
    def grid___add__(self,other):
        print("__add__")
        return self.lit___add__(other)
    torch.FloatTensor.__add__ = grid___add__
    
    torch.FloatTensor.lit_add = torch.FloatTensor.add
    def grid_add(self,other):
        print("add")
        return self.lit_add(other)
    torch.FloatTensor.add = grid_add
    
def hook_torch():
    torch.lit_addmm = torch.addmm
    def grid_addmm(mat, mat1, mat2, beta=1, alpha=1):
        print("addmm")
        return torch.addmm(mat, mat1, mat2, beta = beta, alpha = alpha)

In [3]:
hook_float()

In [4]:
x = torch.FloatTensor([1,2,3,4])

In [5]:
x.add(x)

add



 2
 4
 6
 8
[torch.FloatTensor of size 4]

In [6]:
x + x

__add__
add



 2
 4
 6
 8
[torch.FloatTensor of size 4]

In [32]:
def graph():
    frames = inspect.stack()
    x = Variable(torch.FloatTensor([1,2,3,4]),requires_grad=True)
    y = Variable(torch.FloatTensor([2,3,4,5]),requires_grad=True)
    z = x.add(y)
    z.sum().backward()
    return x.grad, y.grad, frames

In [33]:
graph()

(Variable containing:
  1
  1
  1
  1
 [torch.FloatTensor of size 4], Variable containing:
  1
  1
  1
  1
 [torch.FloatTensor of size 4], [FrameInfo(frame=<frame object at 0x7f8b39eac600>, filename='<ipython-input-32-3c1f715c0d0c>', lineno=2, function='graph', code_context=['    frames = inspect.stack()\n'], index=0),
  FrameInfo(frame=<frame object at 0x7f8b39ed07c8>, filename='<ipython-input-33-76e6a91db885>', lineno=1, function='<module>', code_context=['graph()\n'], index=0),
  FrameInfo(frame=<frame object at 0x55a59652ca98>, filename='/home/jason/anaconda3/envs/openmined/lib/python3.6/site-packages/IPython/core/interactiveshell.py', lineno=2910, function='run_code', code_context=['                exec(code_obj, self.user_global_ns, self.user_ns)\n'], index=0),
  FrameInfo(frame=<frame object at 0x55a59652c828>, filename='/home/jason/anaconda3/envs/openmined/lib/python3.6/site-packages/IPython/core/interactiveshell.py', lineno=2856, function='run_ast_nodes', code_context=['      

In [13]:
y.grad

Variable containing:
 1
 1
 1
 1
[torch.FloatTensor of size 4]

In [4]:
# Notes -- very unstable.
# I think some of the things I'm replacing might have decorators, which could lead to some of the weird behavior
# We need to hook to these methods so that we can send out commands to workers
# Just need to be able to do so reliably
# May also need to disable the __str__ or print functionality on variables/tensors to avoid these RuntimeErrors

def hook_var():
    
    # None of the `__init__` or `new` stuff works yet
    curse(torch._C._VariableBase, "old__init__", torch._C._VariableBase.new)
    #curse(Variable, "old__init__", Variable.new)
    def _new___init__(self, *args):
        self.id = 1
        self.data_addr = 2
        print("init")
        #return self.old__init__(*args)
    #curse(torch._C._VariableBase, "__init__", _new___init__)
    curse(torch._C._VariableBase, "new", _new___init__)
    
    # `add` and `__add__` work sometimes, and even then not as expected
    curse(Variable, "old__add__", Variable.__add__)
    def _new___add__(self, other):
        print("__add__")
        #return self.old__add__(other)
    curse(Variable, "__add__", _new___add__)
   
    curse(torch._C._VariableBase, 'old_add', torch._C._VariableBase.add)
    def _new_add(self, other):
        print("add")
        #return self.old_add(other)
    curse(Variable, 'add', _new_add)
    
    # This works, usually
    torch.old_addmm = torch.addmm
    def _new_addmm(mat, mat1, mat2, beta=1, alpha=1):
        print("addmm")
        #return torch.old_addmm(beta, mat, alpha, mat1, mat2)
    torch.addmm = _new_addmm

In [5]:
def hook_torch():
    torch.tensor = grid_tensor
    torch.zeros = grid_zeros
    hook_var()

    
    #hook_long()

    # anything else we need to do to modify/override torch for the Client globally
    
    pass

In [6]:
hook_torch()

In [7]:
x = torch.tensor([2,2], requires_grad = True)

[2, 2]
None
None
True


In [8]:
x.id

AttributeError: 'Variable' object has no attribute 'id'

In [9]:
x.cuda()

RuntimeError: Overflow when unpacking long

In [10]:
x.add(x)

RuntimeError: Overflow when unpacking long

In [11]:
x + x

RuntimeError: Overflow when unpacking long

In [12]:
torch.addmm(x, x, x)

addmm


Below doesn't work -- can't subclass FloatTensor?

In [1]:
# torch.tensor(data, dtype=None, device=None, requires_grad=False)
# torch.zeros(shape, dtype=None, device=None, requires_grad=False)
# torch.ones(...)
# torch.empty(...)

In [1]:
import torch
from torch.autograd import Variable

In [15]:
class FakeVar(torch.autograd.variable):
    
    def __add__(self, other):
        return self.add(other)
    
    def add(self, other):
        print("add")


In [16]:
torch._C._VariableBase = FakeVar

In [20]:
torch.autograd.Variable = FakeVar

In [8]:
a = torch.tensor((2,2), dtype = torch.float, requires_grad = True)

(2, 2)
torch.float32
True


In [9]:
a.add(a)

__add__
__add__


RuntimeError: Overflow when unpacking long

In [23]:
type(a)

torch.autograd.variable.Variable

In [24]:
type(torch.tensor((2,2), requires_grad = True))

(2, 2)
None
True


torch.autograd.variable.Variable

In [25]:
a + a


 3.1621e-28  9.1370e-41
 3.1621e-28  9.1370e-41
[torch.FloatTensor of size (2,2)]